<a href="https://colab.research.google.com/github/ishika-b/neural-networks/blob/main/nnfl_assn2_q6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from scipy.io import loadmat
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
import math
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix

In [ ]:
if __name__ == '__main__':
  mat_data = loadmat('/content/drive/My Drive/nnfl/nnfl2/data5.mat')
  data = mat_data['x'] #numpy array

  np.random.seed(2)
  np.random.shuffle(data) #shuffling row wise to get a 2148x73 np array

  x = data[:,:-1]
  y = data[:,-1]
  #print(np.unique(y)) 2 class levels : 1 , 0

  for i in range(x.shape[1]):
    mean = np.mean(x[:,i]) 
    std = np.std(x[:,i])
    x[:,i] = [(a-mean)/std for a in x[:,i]]
 
  acc = []
  class_zero_acc = []
  class_one_acc = []

WITH TANH

In [ ]:

  cf = KFold(n_splits=5, shuffle=True)
  fold = 1 
  for train_index, test_index in cf.split(x):
      #print("TRAIN:", train_index.shape, "TEST:", test_index.shape)
      x_train, x_test = x[train_index], x[test_index]
      y_tr, y_test = y[train_index], y[test_index]

      one_hot_y = np.eye(2)
      y_train = np.zeros((y_tr.shape[0], 2))
      for i in range(y_train.shape[0]):
        if (y_tr[i] == 1):
          y_train[i] = one_hot_y[1]
        else: y_train[i] = one_hot_y[0]

      #(m,L) hidden layer matrix
      x_train = np.append(np.ones((x_train.shape[0],1)),x_train,axis=1) 
      random_mat = np.random.standard_normal((x_train.shape[1],10000)) # 73 x num of hidden neurons 
      tempH = np.matmul(x_train,random_mat) #1504x100
      H = np.tanh(tempH)
      W = np.matmul(np.linalg.pinv(H),y_train)

      x_test = np.append(np.ones((x_test.shape[0],1)),x_test,axis=1)
      H_test = np.tanh(np.matmul(x_test,random_mat))
      Y_pred = np.matmul(H_test,W)

      Y_pred = np.argmax(Y_pred,axis = 1)
      score = accuracy_score(y_test, Y_pred)
      print("overall accuracy in ", fold," :",score.mean())
      acc.append(score.mean())

      cm = confusion_matrix(y_test, Y_pred)
      print("class 0 accuracy in ", fold," :", cm[0,0]/(cm[0,0]+cm[0,1]))
      class_zero_acc.append(cm[0,0]/(cm[0,0]+cm[0,1]))
      print("class 1 accuracy in ", fold," :", cm[1,1]/(cm[1,1]+cm[1,0]))
      class_one_acc.append(cm[1,1]/(cm[1,1]+cm[1,0]))

      fold = fold + 1

overall accuracy in  1  : 0.9674418604651163
class 0 accuracy in  1  : 0.9811320754716981
class 1 accuracy in  1  : 0.9541284403669725
overall accuracy in  2  : 0.9534883720930233
class 0 accuracy in  2  : 0.9423076923076923
class 1 accuracy in  2  : 0.963963963963964
overall accuracy in  3  : 0.9697674418604652
class 0 accuracy in  3  : 0.9598214285714286
class 1 accuracy in  3  : 0.9805825242718447
overall accuracy in  4  : 0.958041958041958
class 0 accuracy in  4  : 0.9285714285714286
class 1 accuracy in  4  : 0.9863013698630136
overall accuracy in  5  : 0.9463869463869464
class 0 accuracy in  5  : 0.9457013574660633
class 1 accuracy in  5  : 0.9471153846153846


In [ ]:
from statistics import mean 
print("average overall accuracy", mean(acc))
print("average class 0 accuracy", mean(class_zero_acc))
print("average class 1 accuracy", mean(class_one_acc))

average overall accuracy 0.9590253157695018
average class 0 accuracy 0.9515067964776622
average class 1 accuracy 0.9664183366162359


WITH GAUSSIAN

In [ ]:
def gaussian(z):
  return  (-1/2)*np.exp(z)

In [ ]:
  cf = KFold(n_splits=5, shuffle=True)
  fold = 1 
  for train_index, test_index in cf.split(x):
      #print("TRAIN:", train_index.shape, "TEST:", test_index.shape)
      x_train, x_test = x[train_index], x[test_index]
      y_tr, y_test = y[train_index], y[test_index]

      one_hot_y = np.eye(2)
      y_train = np.zeros((y_tr.shape[0], 2))
      for i in range(y_train.shape[0]):
        if (y_tr[i] == 1):
          y_train[i] = one_hot_y[1]
        else: y_train[i] = one_hot_y[0]

      #(m,L) hidden layer matrix
      x_train = np.append(np.ones((x_train.shape[0],1)),x_train,axis=1) 
      random_mat = np.random.standard_normal((x_train.shape[1],7000)) # 73 x num of hidden neurons 

      tempH = np.zeros((x_train.shape[0], random_mat.shape[1]))

      b = abs(np.random.standard_normal())
  
      for i in range(x_train.shape[0]):
        for j in range(random_mat.shape[1]):
            tempH[i][j] = np.linalg.norm(x_train[i,:] - random_mat[:,j]) #(x_train = 1504, 73)(random_mat= 73x1000)= (1504x100)
      H = np.exp(b*tempH)
      W = np.matmul(np.linalg.pinv(H),y_train)

      x_test = np.append(np.ones((x_test.shape[0],1)),x_test,axis=1) #430,73

      tempH_test = np.zeros((x_test.shape[0], random_mat.shape[1]))

      for i in range(x_test.shape[0]):#430
        for j in range(random_mat.shape[1]):#7000
            tempH_test[i][j] = np.linalg.norm(x_test[i,:] - random_mat[:,j]) #(x_train = 1504, 73)(random_mat= 73x1000)= (1504x100)
      H_test = np.exp(b*tempH_test)
      Y_pred = np.matmul(H_test,W)

      Y_pred = np.argmax(Y_pred,axis = 1)
      score = accuracy_score(y_test, Y_pred)
      print("overall accuracy in ", fold," :",score.mean())
      acc.append(score.mean())

      cm = confusion_matrix(y_test, Y_pred)
      print("class 0 accuracy in ", fold," :", cm[0,0]/(cm[0,0]+cm[0,1]))
      class_zero_acc.append(cm[0,0]/(cm[0,0]+cm[0,1]))
      print("class 1 accuracy in ", fold," :", cm[1,1]/(cm[1,1]+cm[1,0]))
      class_one_acc.append(cm[1,1]/(cm[1,1]+cm[1,0]))

      fold = fold + 1

overall accuracy in  1  : 0.49534883720930234
class 0 accuracy in  1  : 1.0
class 1 accuracy in  1  : 0.0
overall accuracy in  2  : 0.9279069767441861
class 0 accuracy in  2  : 0.9140271493212669
class 1 accuracy in  2  : 0.9425837320574163
overall accuracy in  3  : 0.43953488372093025
class 0 accuracy in  3  : 0.11304347826086956
class 1 accuracy in  3  : 0.815
overall accuracy in  4  : 0.4405594405594406
class 0 accuracy in  4  : 0.2
class 1 accuracy in  4  : 0.6607142857142857
overall accuracy in  5  : 0.41025641025641024
class 0 accuracy in  5  : 0.24271844660194175
class 1 accuracy in  5  : 0.5650224215246636


In [ ]:
print("average overall accuracy", np.mean(acc))
print("average class 0 accuracy", np.mean(class_zero_acc))
print("average class 1 accuracy", np.mean(class_one_acc))

average overall accuracy 0.5723621844956526
average class 0 accuracy 0.6780970347113443
average class 1 accuracy 0.46654794139155303
